## Predict Spend

In [12]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [13]:
import os
import sys
import numpy as np
from pyspark.sql import functions as F
import tensorflow_probability as tfp
import tensorflow as tf

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('mining/')[0])
# from rezaware.modules.etl.loader import sparkRDBM as db
from rezaware.modules.etl.loader import sparkFile as file
# from rezaware.modules.etl.loader import __propAttr__ as attr

''' restart initiate classes '''
if debug:
    import importlib
    # db = importlib.reload(db)
    file=importlib.reload(file)
    # attr=importlib.reload(attr)

__desc__ = "read and write BigQuery dataset for hypothese testing"
# clsSDB = db.dataWorkLoads(
#     desc=__desc__,
#     db_type = 'bigquery',
#     db_driver=None,
#     db_hostIP=None,
#     db_port = None,
#     db_name = None,
#     db_schema='combined_data_facebook_ads',
#     spark_partitions=None,
#     spark_format = 'bigquery',
#     spark_save_mode=None,
#     # spark_jar_dir = _jar,
# )
clsFile = file.dataWorkLoads(
    desc = "optimizing action_type budgets for an ad",
    store_mode='local-fs',
    store_root=proj_dir.split('mining/')[0],
    jar_dir=None,
)
# if clsSDB.session:
#     clsSDB._session.stop
print("\n%s class initialization and load complete!" % __desc__)

All functional SPARKFILE-libraries in LOADER-package of ETL-module imported successfully!
All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
__propAttr__ Class initialization complete
sparkFile Class initialization complete

read and write BigQuery dataset for hypothese testing class initialization and load complete!


## Load Data

In [14]:
options = {
    "inferSchema":True,
    "header":True,
    "delimiter":",",
    "pathGlobFilter":'*.csv',
    "recursiveFileLookup":True,
}

sdf=clsFile.read_files_to_dtype(
    as_type = "SPARK",      # optional - define the data type to return
    folder_path="mining/data/budget/",  # optional - relative path, w.r.t. self.storeRoot
        file_name="complete-60-accounts.csv",  # optional - name of the file to read
        file_type=None,  # optional - read all the files of same type
        **options,
)
print("Loaded %d rows" % sdf.count())
sdf.printSchema()

25/02/16 13:38:59 WARN Utils: Your hostname, Waidy-Think-Three resolves to a loopback address: 127.0.1.1; using 192.168.2.82 instead (on interface enp0s25)
25/02/16 13:38:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/02/16 13:39:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/16 13:39:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/16 13:39:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
                                                                                

Loaded 61513 rows
root
 |-- account_id: long (nullable = true)
 |-- ad_id: long (nullable = true)
 |-- adset_id: long (nullable = true)
 |-- campaign_id: long (nullable = true)
 |-- updated_time: date (nullable = true)
 |-- impressions: integer (nullable = true)
 |-- frequency: double (nullable = true)
 |-- reach: integer (nullable = true)
 |-- CTR: double (nullable = true)
 |-- CPC: double (nullable = true)
 |-- CPM: double (nullable = true)
 |-- ROAS: double (nullable = true)
 |-- social_spend: double (nullable = true)
 |-- spend: double (nullable = true)
 |-- click: integer (nullable = true)
 |-- auction_bid: integer (nullable = true)
 |-- goal: string (nullable = true)
 |-- purchase_value: double (nullable = true)
 |-- account_currency: string (nullable = true)
 |-- purchase: double (nullable = true)
 |-- fb_pixel_view_value: double (nullable = true)
 |-- fb_pixel_purchase_value: double (nullable = true)
 |-- fb_pixel_add_to_cart_value: double (nullable = true)
 |-- mobile_app_purc

In [22]:
_data_df= sdf.select('spend','CPC', 'CPM', 'CTR', 'ROAS').dropna().toPandas()
print(_data_df.shape)
_data_df = _data_df[_data_df['spend']<=\
                    _data_df['spend'].mean()+2*_data_df['spend'].std()]
_data_df = _data_df[_data_df['CTR']<=\
                    _data_df['CTR'].mean()+2*_data_df['CTR'].std()]
_data_df = _data_df[_data_df['CPC']<=\
                    _data_df['CPC'].mean()+2*_data_df['CPC'].std()]
_data_df = _data_df[_data_df['ROAS']<=\
                    _data_df['ROAS'].mean()+2*_data_df['ROAS'].std()]
_data_df = _data_df[_data_df['CPM']<=\
                    _data_df['CPM'].mean()+2*_data_df['CPM'].std()]
# _data_df = _data_df[_data_df['spend']<=4000]

# _data_df = _data_df.sample(n=1000)

print("data shape:", _data_df.shape)
print("data quartile  :\n", _data_df.describe())

(26193, 5)
data shape: (21771, 5)
data quartile  :
               spend           CPC           CPM           CTR          ROAS
count  21771.000000  21771.000000  21771.000000  21771.000000  21771.000000
mean     653.172401      4.070958     83.640293      3.125367      4.484789
std      867.888233      4.019480     61.742344      2.342454     19.268487
min        0.060000      0.000000      2.310524      0.000000      0.000332
25%       57.765000      1.184179     35.298006      1.374857      0.650808
50%      310.560000      2.478642     63.228397      2.338685      1.412899
75%      869.595000      5.667179    117.630144      4.281987      3.164864
max     4371.290000     19.852045    268.761782     11.179441    571.764706


In [36]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

tfd = tfp.distributions
tfb = tfp.bijectors

# Example historic data (replace with your actual data)
# Each row represents a historic data point: [Spend, CPC, CPM, CTR, ROAS]
# historic_data = np.array([
#     [100, 0.5, 10, 0.02, 4.0],
#     [200, 0.6, 12, 0.03, 3.5],
#     [300, 0.7, 15, 0.04, 3.0],
#     # Add more data points here
# ])
historic_data = np.array(_data_df)

# Split the data into input (Spend) and outputs (CPC, CPM, CTR, ROAS)
spend = historic_data[:, 0].astype(np.float32)  # Ensure float32 for TensorFlow
cpc = historic_data[:, 1].astype(np.float32)
cpm = historic_data[:, 2].astype(np.float32)
ctr = historic_data[:, 3].astype(np.float32)
roas = historic_data[:, 4].astype(np.float32)

# Scale the Spend data to improve numerical stability
spend_mean = np.mean(spend)
spend_std = np.std(spend)
spend_scaled = (spend - spend_mean) / spend_std

# Define the probabilistic model
def model():
    # Priors for the parameters
    alpha_cpc = yield tfd.Normal(loc=0.0, scale=1.0, name="alpha_cpc")
    beta_cpc = yield tfd.Normal(loc=0.0, scale=1.0, name="beta_cpc")
    
    alpha_cpm = yield tfd.Normal(loc=0.0, scale=1.0, name="alpha_cpm")
    beta_cpm = yield tfd.Normal(loc=0.0, scale=1.0, name="beta_cpm")
    
    alpha_ctr = yield tfd.Normal(loc=0.0, scale=1.0, name="alpha_ctr")
    beta_ctr = yield tfd.Normal(loc=0.0, scale=1.0, name="beta_ctr")
    
    alpha_roas = yield tfd.Normal(loc=0.0, scale=1.0, name="alpha_roas")
    beta_roas = yield tfd.Normal(loc=0.0, scale=1.0, name="beta_roas")
    
    # Likelihoods
    cpc_pred = yield tfd.Normal(loc=alpha_cpc + beta_cpc * spend_scaled, scale=0.1, name="cpc_pred")
    cpm_pred = yield tfd.Normal(loc=alpha_cpm + beta_cpm * spend_scaled, scale=0.1, name="cpm_pred")
    ctr_pred = yield tfd.Normal(loc=alpha_ctr + beta_ctr * spend_scaled, scale=0.01, name="ctr_pred")
    roas_pred = yield tfd.Normal(loc=alpha_roas + beta_roas * spend_scaled, scale=0.1, name="roas_pred")

# Define the joint distribution
joint_distribution = tfd.JointDistributionCoroutineAutoBatched(model)

# Define the log probability function
def log_prob(alpha_cpc, beta_cpc, alpha_cpm, beta_cpm, alpha_ctr, beta_ctr, alpha_roas, beta_roas):
    return joint_distribution.log_prob(
        alpha_cpc=alpha_cpc,
        beta_cpc=beta_cpc,
        alpha_cpm=alpha_cpm,
        beta_cpm=beta_cpm,
        alpha_ctr=alpha_ctr,
        beta_ctr=beta_ctr,
        alpha_roas=alpha_roas,
        beta_roas=beta_roas,
        cpc_pred=cpc,
        cpm_pred=cpm,
        ctr_pred=ctr,
        roas_pred=roas,
    )

# Use MCMC to sample from the posterior
num_results = 2000  # Increase the number of samples
num_burnin_steps = 1000  # Increase the burn-in steps

# Initialize the chain with reasonable values
initial_chain_state = [
    tf.constant(0.5),  # alpha_cpc
    tf.constant(0.1),  # beta_cpc
    tf.constant(10.0),  # alpha_cpm
    tf.constant(0.1),  # beta_cpm
    tf.constant(0.02),  # alpha_ctr
    tf.constant(0.01),  # beta_ctr
    tf.constant(4.0),  # alpha_roas
    tf.constant(0.1),  # beta_roas
]

# Define the MCMC kernel
hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=log_prob,
    num_leapfrog_steps=10,
    step_size=0.05,  # Adjusted step size
)

# Run the MCMC sampler
@tf.function
def run_chain():
    return tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burnin_steps,
        current_state=initial_chain_state,
        kernel=hmc_kernel,
        trace_fn=None,
    )

# Run the chain and get samples
samples = run_chain()

# Extract the posterior samples
alpha_cpc_samples, beta_cpc_samples, alpha_cpm_samples, beta_cpm_samples, \
alpha_ctr_samples, beta_ctr_samples, alpha_roas_samples, beta_roas_samples = samples

# Predict expected values for a given Spend
def predict(spend_value):
    # Scale the input Spend value
    spend_value_scaled = (spend_value - spend_mean) / spend_std
    
    # Compute predictions
    cpc_pred = np.mean(alpha_cpc_samples + beta_cpc_samples * spend_value_scaled)
    cpm_pred = np.mean(alpha_cpm_samples + beta_cpm_samples * spend_value_scaled)
    ctr_pred = np.mean(alpha_ctr_samples + beta_ctr_samples * spend_value_scaled)
    roas_pred = np.mean(alpha_roas_samples + beta_roas_samples * spend_value_scaled)
    
    return cpc_pred, cpm_pred, ctr_pred, roas_pred


Expected CPC for Spend=250: 0.45354458689689636
Expected CPM for Spend=250: 9.953546524047852
Expected CTR for Spend=250: 0.015354453586041927
Expected ROAS for Spend=250: 3.9535443782806396


In [42]:
# Example prediction for a given Spend
spend_value = 6.44  # Replace with your desired Spend value
cpc_pred, cpm_pred, ctr_pred, roas_pred = predict(spend_value)

print(f"Expected CPC for Spend={spend_value}: {cpc_pred}")
print(f"Expected CPM for Spend={spend_value}: {cpm_pred}")
print(f"Expected CTR for Spend={spend_value}: {ctr_pred}")
print(f"Expected ROAS for Spend={spend_value}: {roas_pred}")

Expected CPC for Spend=6.44: 0.42548036575317383
Expected CPM for Spend=6.44: 9.9254789352417
Expected CTR for Spend=6.44: 0.012548033148050308
Expected ROAS for Spend=6.44: 3.9254798889160156


In [ ]:
643.58975     4.068199    83.871559     3.038284     3.730636

In [28]:
_data_df[_data_df['ROAS']>4]

,spend,CPC,CPM,CTR,ROAS
0,6.44,0.920000,11.398230,1.238938,18.636646
1,5.00,1.000000,13.227513,1.322751,18.190000
2,25.84,1.845714,29.397042,1.592719,9.854876
3,16.29,1.086000,10.745383,0.989446,5.831799
4,63.54,1.245882,55.933099,4.489437,8.462386
...,...,...,...,...,...
26080,4184.25,8.645145,261.990483,3.030493,4.124366
26179,28.53,1.188750,37.989348,3.195739,5.213810
26181,49.69,1.014082,41.134106,4.056291,5.895150
26189,19.59,0.932857,36.892655,3.954802,7.918836
